# Importamos librerias

In [31]:
import pandas as pd
import numpy as np
import os
import snowflake.connector
import json
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
pd.options.mode.chained_assignment = None

In [32]:
while True:
    try:

        f = open('credentials.json')
        data_pass = json.load(f)

        pass_ = input("INGRESAR PASSCODE:")

        ctx = snowflake.connector.connect(
            user = data_pass['snow']['USER'],
            password = data_pass['snow']['PASS'],
            account = data_pass['snow']['ACCOUNT'],
            passcode = pass_,
            database = 'SANDBOX_PLUS',
            schema = 'DWH'
        )

        cursor = ctx.cursor()

        print('Connected')

        break

    except:
        print('Incorrect Password - provide again')

Connected


# Info Ofertatas

In [33]:
ofertata_1_orines = tuple(['1000054862', '1000054863', '1000053389', '1000353034', '1000353035', '1000057303', '1000057084', '1000408185', '1000047733', '1000103472', '1000103473', '1000103474', '1000103475', '1000103476', '1000103477', '1000454069', '1000099231', '1000099229', '1000099233', '1000099234', '1000420775', '1000455579', '1000041807', '1000041820', '1000041821', '1000041822', '1000041825'])
ofertata_1_inicio = '2024-01-04'
ofertata_1_fin = '2024-01-10'
ofertata_1_label = 'Ofertata 1'

In [34]:
ofertata_2_orines = tuple(['1000050223', '1000052025', '1000052026', '1000050604', '1000051793', '1000050430', '1000052904', '1000055018', '1000054557', '1000054561', '1000054554', '1000115080', '1000050183', '1000057475', '1000157003', '1000300431', '1000056430', '1000056460', '1000056444', '1000047820', '1000047821', '1000293393', '1000293392', '1000293399', '1000047464', '1000047463', '1000059577', '1000099194', '1000099195', '1000099197', '1000099200', '1000025827', '1000361491', '1000041840', '1000041838', '1000041824', '1000041839', '1000037657'])
ofertata_2_inicio = '2024-01-11'
ofertata_2_fin = '2024-01-17'
ofertata_2_label = 'Ofertata 2'

In [35]:
sets_of_data = []

# Ofertata 1
set_1 = {
    'label': ofertata_1_label,
    'inicio': ofertata_1_inicio,
    'fin': ofertata_1_fin,
    'orines': ofertata_1_orines
}
sets_of_data.append(set_1)

# Ofertata 2
set_2 = {
    'label': ofertata_2_label,
    'inicio': ofertata_2_inicio,
    'fin': ofertata_2_fin,
    'orines': ofertata_2_orines
}
sets_of_data.append(set_2)
print(sets_of_data)

[{'label': 'Ofertata 1', 'inicio': '2024-01-04', 'fin': '2024-01-10', 'orines': ('1000054862', '1000054863', '1000053389', '1000353034', '1000353035', '1000057303', '1000057084', '1000408185', '1000047733', '1000103472', '1000103473', '1000103474', '1000103475', '1000103476', '1000103477', '1000454069', '1000099231', '1000099229', '1000099233', '1000099234', '1000420775', '1000455579', '1000041807', '1000041820', '1000041821', '1000041822', '1000041825')}, {'label': 'Ofertata 2', 'inicio': '2024-01-11', 'fin': '2024-01-17', 'orines': ('1000050223', '1000052025', '1000052026', '1000050604', '1000051793', '1000050430', '1000052904', '1000055018', '1000054557', '1000054561', '1000054554', '1000115080', '1000050183', '1000057475', '1000157003', '1000300431', '1000056430', '1000056460', '1000056444', '1000047820', '1000047821', '1000293393', '1000293392', '1000293399', '1000047464', '1000047463', '1000059577', '1000099194', '1000099195', '1000099197', '1000099200', '1000025827', '1000361491

# PLUS

## Venta y GB1

In [36]:
query = '''
SELECT
    FV.TICKET,
    FFM.SOCI_SOCI_ID,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) AS VENTA_PLUS,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) - SUM(FV.VNTA_COSTO_PROM_POND * FV.VNTA_UNIDADES) AS GB1_PLUS
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FFM.TICKET = FV.TICKET
WHERE
    LAA.ORIN IN {orines_sql}
    AND FV.TIEM_DIA_ID BETWEEN '{inicio_sql}' AND '{fin_sql}'
GROUP BY
    ALL
'''

plus = pd.DataFrame()

for set_data in sets_of_data:
    orines = set_data['orines']
    inicio = set_data['inicio']
    fin = set_data['fin']
    label = set_data['label']

    cursor.execute(query.format(
        orines_sql = orines,
        inicio_sql = inicio,
        fin_sql = fin
    ))
    df_aux = cursor.fetch_pandas_all()
    df_aux['TIPO'] = label
    plus = pd.concat([df_aux, plus])
plus.head(2)

,TICKET,SOCI_SOCI_ID,VENTA_PLUS,GB1_PLUS,TIPO
0,2024011197938795,774131,210.11,37.981800,Ofertata 2
1,202401111851068952,719100,352.80,49.738608,Ofertata 2


## Ticket Mediano

In [37]:
query = '''
WITH TICKETS AS
    (
    SELECT
        DISTINCT FV.TICKET
    FROM
        MSTRDB.DWH.FT_VENTAS AS FV
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FFM.TICKET = FV.TICKET
    WHERE
        LAA.ORIN IN {orines_sql}
        AND FV.TIEM_DIA_ID BETWEEN '{inicio_sql}' AND '{fin_sql}'
    )

SELECT
    MEDIAN(VENTAS) AS TICKET_MEDIANO_PLUS
FROM
    (
    SELECT
        TICKET,
        SUM(VNTA_IMPORTE_SIN_IVA) AS VENTAS
    FROM
        MSTRDB.DWH.FT_VENTAS
    WHERE
        TICKET IN (SELECT * FROM TICKETS)
    GROUP BY
        ALL
    )
'''

plus_ticket_mediano = pd.DataFrame()

for set_data in sets_of_data:
    orines = set_data['orines']
    inicio = set_data['inicio']
    fin = set_data['fin']
    label = set_data['label']

    cursor.execute(query.format(
        orines_sql = orines,
        inicio_sql = inicio,
        fin_sql = fin
    ))
    df_aux = cursor.fetch_pandas_all()
    df_aux['TIPO'] = label
    plus_ticket_mediano = pd.concat([df_aux, plus_ticket_mediano])
plus_ticket_mediano.head(2)

,TICKET_MEDIANO_PLUS,TIPO
0,737.98,Ofertata 2
0,708.28,Ofertata 1


## Consolidado

In [38]:
plus_2 = plus.groupby('TIPO').agg(
    {'VENTA_PLUS':'sum',
    'GB1_PLUS':'sum',
     'TICKET':'nunique',
     'SOCI_SOCI_ID':'nunique'
     }).reset_index()

plus_2.rename({'TICKET':'TICKETS_PLUS', 'SOCI_SOCI_ID':'CLIENTES'}, axis = 1, inplace = True)

plus_2 = plus_2.merge(plus_ticket_mediano, on = 'TIPO', how = 'left')
plus_2['TICKET_MEDIANO_PLUS'] = round(plus_2['TICKET_MEDIANO_PLUS'], 0).astype(int)

plus_2.sort_values(by = 'TIPO', inplace = True)
plus_2

,TIPO,VENTA_PLUS,GB1_PLUS,TICKETS_PLUS,CLIENTES,TICKET_MEDIANO_PLUS
0,Ofertata 1,5406127.32,9.327527e+05,43955,38449,708
1,Ofertata 2,9652994.87,1.303648e+06,61711,53059,738


# Total

In [39]:
query = '''
SELECT
    FV.TICKET,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) AS VENTA_TOTAL,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) - SUM(FV.VNTA_COSTO_PROM_POND * FV.VNTA_UNIDADES) AS GB1_TOTAL
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
WHERE
    LAA.ORIN IN {orines_sql}
    AND FV.TIEM_DIA_ID BETWEEN '{inicio_sql}' AND '{fin_sql}'
GROUP BY
    ALL
'''

total = pd.DataFrame()

for set_data in sets_of_data:
    orines = set_data['orines']
    inicio = set_data['inicio']
    fin = set_data['fin']
    label = set_data['label']

    cursor.execute(query.format(
        orines_sql = orines,
        inicio_sql = inicio,
        fin_sql = fin
    ))
    df_aux = cursor.fetch_pandas_all()
    df_aux['TIPO'] = label
    total = pd.concat([df_aux, total])
total.head(2)

,TICKET,VENTA_TOTAL,GB1_TOTAL,TIPO
0,20240112551522566,127.05,-30.9962,Ofertata 2
1,20240112230474066,172.13,39.2054,Ofertata 2


## Ticket Mediano

In [42]:
query = '''
WITH TICKETS AS
    (
    SELECT
        DISTINCT FV.TICKET
    FROM
        MSTRDB.DWH.FT_VENTAS AS FV
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    WHERE
        LAA.ORIN IN {orines_sql}
        AND FV.TIEM_DIA_ID BETWEEN '{inicio_sql}' AND '{fin_sql}'
    )

SELECT
    MEDIAN(VENTAS) AS TICKET_MEDIANO_TOTAL
FROM
    (
    SELECT
        TICKET,
        SUM(VNTA_IMPORTE_SIN_IVA) AS VENTAS
    FROM
        MSTRDB.DWH.FT_VENTAS
    WHERE
        TICKET IN (SELECT * FROM TICKETS)
    GROUP BY
        ALL
    )
'''

total_ticket_mediano = pd.DataFrame()

for set_data in sets_of_data:
    orines = set_data['orines']
    inicio = set_data['inicio']
    fin = set_data['fin']
    label = set_data['label']

    cursor.execute(query.format(
        orines_sql = orines,
        inicio_sql = inicio,
        fin_sql = fin
    ))
    df_aux = cursor.fetch_pandas_all()
    df_aux['TIPO'] = label
    total_ticket_mediano = pd.concat([df_aux, total_ticket_mediano])
total_ticket_mediano.head(2)

,TICKET_MEDIANO_TOTAL,TIPO
0,733.08,Ofertata 2
0,702.31,Ofertata 1


## Consolidado

In [44]:
total_2 = total.groupby('TIPO').agg(
    {'VENTA_TOTAL':'sum',
    'GB1_TOTAL':'sum',
     'TICKET':'nunique'
     }).reset_index()

total_2.rename({'TICKET':'TICKETS_TOTALES'}, axis = 1, inplace = True)

total_2 = total_2.merge(total_ticket_mediano, on = 'TIPO', how = 'left')
total_2['TICKET_MEDIANO_TOTAL'] = round(total_2['TICKET_MEDIANO_TOTAL'], 0).astype(int)

total_2.sort_values(by = 'TIPO', inplace = True)
total_2

,TIPO,VENTA_TOTAL,GB1_TOTAL,TICKETS_TOTALES,TICKET_MEDIANO_TOTAL
0,Ofertata 1,6102422.52,1.151558e+06,48052,702
1,Ofertata 2,10338106.28,1.479975e+06,65977,733


# Consolidado

In [45]:
plus_2

,TIPO,VENTA_PLUS,GB1_PLUS,TICKETS_PLUS,CLIENTES,TICKET_MEDIANO_PLUS
0,Ofertata 1,5406127.32,9.327527e+05,43955,38449,708
1,Ofertata 2,9652994.87,1.303648e+06,61711,53059,738


In [48]:
consolidado = total_2.merge(plus_2, on = 'TIPO', how = 'left')
consolidado = consolidado[['TIPO', 'VENTA_TOTAL', 'VENTA_PLUS', 'GB1_TOTAL',
                           'GB1_PLUS', 'TICKETS_TOTALES', 'TICKETS_PLUS',
                           'TICKET_MEDIANO_TOTAL', 'TICKET_MEDIANO_PLUS',
                           'CLIENTES']]
consolidado.to_excel('Ofertata 1 vs 2.xlsx', index = False)

# Fin